## Prep

In [14]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# ------- commutation matrix -------
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]

# ------- matrix A(Sigma) -------
def create_A_Sigma(cov: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))

# ------- matrix A (intervention) -------
def create_A(A_Sigma: np.ndarray, mean: np.ndarray, p: int) -> np.ndarray:
    identity = mean[0] * np.eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * np.eye(p)))
    return np.vstack((A_Sigma, identity))

# ------- Gram matrix -------
def gram(A: np.ndarray) -> np.ndarray:
    return np.matmul(np.transpose(A), A)

# ------- vector g -------
def vec_g(A: np.ndarray, vec_C: np.ndarray) -> np.ndarray:
    return - np.dot(A, vec_C)

# ------- irrepresentability condition -------
def irrep_cond(A: np.ndarray, supp: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], supp] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(supp, supp)])
        return np.linalg.norm(np.matmul(matrix_1, matrix_2), ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

# ------- weak irrepresentability condition -------
def weak_irrep_cond(M: np.ndarray, A: np.ndarray, supp: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], supp] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(supp, supp)])
        sign_vec = np.sign(M.flatten(order="F"))[supp]
        return np.linalg.norm(np.matmul(matrix_1, matrix_2) @ sign_vec, ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

num_exp = 100000 # number of true matrices per case
#num_exp = 10 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix
b = 2
reg_param = 0.001

## Experiments

### Ex. 1
<img src="Graphics/graph29.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 3], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 6, 8, 10, 15])
supp_compl = np.array([2, 3, 4, 7, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_1 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_1 = experiment_cycle_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_1.to_pickle("Experiments/cycle_1.pkl")

### Ex. 2
<img src="Graphics/graph30.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [1, 3], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 6, 8, 10, 15])
supp_compl = np.array([2, 4, 7, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_2 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_2 = experiment_cycle_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_2.to_pickle("Experiments/cycle_2.pkl")

### Ex. 3
<img src="Graphics/graph31.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 6, 8, 10, 12, 15])
supp_compl = np.array([2, 3, 4, 7, 9, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_3 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_3 = experiment_cycle_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_3.to_pickle("Experiments/cycle_3.pkl")

### Ex. 4
<img src="Graphics/graph32.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 3], [1, 0], [1, 2], [2, 0], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 2, 5, 7, 9, 10, 12, 15])
supp_compl = np.array([1, 3, 4, 6, 8, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_4 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_4 = experiment_cycle_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_4.to_pickle("Experiments/cycle_4.pkl")

### Ex. 5
<img src="Graphics/graph33.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 6, 7, 8, 10, 15])
supp_compl = np.array([2, 4, 9, 11, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_5 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_5 = experiment_cycle_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_5.to_pickle("Experiments/cycle_5.pkl")

### Ex. 6
<img src="Graphics/graph34.png" alt="Drawing" style="width: 150px;"/>

In [15]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 6, 7, 8, 10, 12, 15])
supp_compl = np.array([2, 3, 4, 9, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_6 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_6 = experiment_cycle_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_6.to_pickle("Experiments/cycle_6.pkl")

### Ex. 7
<img src="Graphics/graph36.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 7, 8, 9, 10, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 11, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_7 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_7 = experiment_cycle_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_7.to_pickle("Experiments/cycle_7.pkl")

### Ex. 8
<img src="Graphics/graph37.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [2, 1], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 6, 7, 8, 10, 11, 15])
supp_compl = np.array([2, 4, 9, 12, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_8 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_8 = experiment_cycle_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_8.to_pickle("Experiments/cycle_8.pkl")

### Ex. 9
<img src="Graphics/graph38.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 3], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 5, 6, 7, 10, 11, 12, 15])
supp_compl = np.array([3, 4, 8, 9, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_9 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_9 = experiment_cycle_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_9.to_pickle("Experiments/cycle_9.pkl")

### Ex. 10
<img src="Graphics/graph39.png" alt="Drawing" style="width: 150px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 3], [2, 0], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 2, 4, 5, 6, 7, 10, 11, 12, 15])
supp_compl = np.array([1, 3, 8, 9, 13, 14])

C = 2 * np.eye(p)
vec_C = C.flatten(order="F")

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 4), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

temp2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
temp3 = pd.DataFrame(np.empty(shape=(num_exp * p, 1), dtype=object), columns=["Estimated M"])

for row in temp2.itertuples():
    index = row.Index
    true_M = temp2.iloc[index, 0]
    unit_v = np.eye(p)[:, temp2.iloc[index, 4]]

    c = np.hstack((vec_C, - b * unit_v))

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)
    emp_A = np.float64(emp_A)

    irrep_condition = irrep_cond(A=emp_A, supp=support, supp_compl=supp_compl)
    weak_irrep_condition = weak_irrep_cond(M=true_M, A=emp_A, supp=support, supp_compl=supp_compl)
    temp2.iloc[index, 1] = (0 <= irrep_condition < 1)
    temp2.iloc[index, 2] = (0 <= weak_irrep_condition <= 1)

    direct_lasso = linear_model.Lasso(alpha=reg_param, max_iter=5000)
    direct_lasso.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)
    temp3.iloc[index, 0] = est_M.round(3)
    temp2.iloc[index, 3] = frob

experiment_cycle_10 = pd.concat([temp2, temp3], axis=1)
experiment_cycle_10 = experiment_cycle_10[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled", "Performance lasso", "Estimated M"]]
experiment_cycle_10.to_pickle("Experiments/cycle_10.pkl")